In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-books-reviews/books_data.csv
/kaggle/input/amazon-books-reviews/Books_rating.csv


# **1- Importing Required Libraries**

In [2]:
%%capture
!pip install -U sentence-transformers
!pip uninstall googletrans -y
!pip install googletrans==4.0.0rc1
!pip install nmslib

In [3]:
import torch
import nmslib
import googletrans
from googletrans import Translator
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# **2- Load Data**

In [4]:
book_df = pd.read_csv('/kaggle/input/amazon-books-reviews/books_data.csv')

book_df.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [5]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [6]:
missing_values_count = book_df.isnull().sum()
missing_values_count

Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64

In [7]:
book_df = book_df.drop(columns = ['image', 'previewLink', 'publishedDate', 'infoLink', 'ratingsCount'])
book_df = book_df.dropna()
book_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121139 entries, 1 to 212402
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Title        121139 non-null  object
 1   description  121139 non-null  object
 2   authors      121139 non-null  object
 3   publisher    121139 non-null  object
 4   categories   121139 non-null  object
dtypes: object(5)
memory usage: 5.5+ MB


# **3- Text Pre-processing with HuggingFace Transformer**

In [8]:
subset = book_df.sample(frac=0.01)
subset.reset_index(inplace = True,drop = True)
subset.shape

(1211, 5)

In [9]:
model = SentenceTransformer('distilbert-base-uncased')

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
embeddings = model.encode(subset['description'], convert_to_tensor=True)
embeddings 

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

tensor([[-0.1337, -0.1119,  0.2863,  ...,  0.0942,  0.1479, -0.0883],
        [-0.0159,  0.5180,  0.1551,  ..., -0.1405,  0.2042, -0.3303],
        [-0.2405,  0.3381, -0.2986,  ..., -0.3286, -0.1323, -0.0190],
        ...,
        [-0.2490,  0.0713,  0.1309,  ..., -0.1512,  0.0878, -0.0556],
        [ 0.0063,  0.2413,  0.0664,  ...,  0.0343,  0.3385,  0.0191],
        [-0.2393,  0.1386, -0.2121,  ..., -0.1355, -0.0940, -0.1882]],
       device='cuda:0')

In [11]:
print(len(embeddings[0]))

768


In [12]:
subset['vectors'] = np.array(embeddings.cpu()).tolist()
subset.head(1)

,Title,description,authors,publisher,categories,vectors
0,Disney's Kim Possible: Pick a Villain - Badica...,She can do anything-and you can help her! Badi...,['Jim Pascoe'],Volo,['Juvenile Fiction'],"[-0.1336873322725296, -0.1118694618344307, 0.2..."


# **4- Creating a Search Index**

## **4-1- Initialize a new index passing 'hnsw' as the method and 'cosinesimil' as the space**

In [13]:
model_index = nmslib.init(method='hnsw', space='cosinesimil')

## **4-2- Add the embeddings to the index using addDataPointBatch() method.**

In [14]:
model_index.addDataPointBatch(embeddings.cpu())

1211

## **4-3- Create the index with the data points using createIndex method()**

In [15]:
model_index.createIndex({'post': 2})

In [16]:
subset.head(2)

,Title,description,authors,publisher,categories,vectors
0,Disney's Kim Possible: Pick a Villain - Badica...,She can do anything-and you can help her! Badi...,['Jim Pascoe'],Volo,['Juvenile Fiction'],"[-0.1336873322725296, -0.1118694618344307, 0.2..."
1,Off the Wall Museum Guides for Kids: Modern Art,Presents an illustrated look at contemporary a...,"['Jacky Klein', 'Suzy Klein']",The Museum of Modern Art,['Juvenile Nonfiction'],"[-0.015926459804177284, 0.5180180072784424, 0...."


# **5- Creating a Search Function**

In [17]:
def find_best_book(data, q_input):
    
    if data is not None and q_input is not None:
        subset = data.copy()
        query = model.encode([q_input], convert_to_tensor=True)
        ids, distances = model_index.knnQuery(query.cpu(), k=20)
        
    best_fits = []
    for i, j in zip(ids, distances):
            
          best_fits.append({'Title':subset.Title.values[i]
                        , 'description' : subset.description.values[i]
                        , 'authors' : subset.authors.values[i]
                        , 'publisher': subset.publisher.values[i]
                        , 'categories': subset.categories.values[i]
                        , 'distance': j
                       })
    return pd.DataFrame(best_fits)

In [18]:
translator = Translator()
input_text = "کتاب هنری و کمدی "
input_text = translator.translate(input_text, dest='en')
print("input text : ",input_text.text)

input text :  Comedy


In [19]:
most_similar=find_best_book(subset, input_text.text)
most_similar.head(3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,Title,description,authors,publisher,categories,distance
0,Music Copyright for the New Millenium,General Reference,['James Frankel'],Hal Leonard Corporation,['Music'],0.169471
1,Cubase Power!,Manuals,['Brian McConnon'],Hal Leonard Corporation,['Music'],0.194342
2,True Prayer: An Invitation to Christian Spirit...,True Prayer,['Kenneth Leech'],"Church Publishing, Inc.",['Religion'],0.195517


In [20]:
for i in range(0,3):
    print(most_similar['description'][i][0:-1])
    print("distance:",most_similar['distance'][i])

General Referenc
distance: 0.16947132
Manual
distance: 0.19434154
True Praye
distance: 0.1955167


In [21]:
output_fa = translator.translate(most_similar['Title'][0], src='auto', dest='fa')
print('title : ',most_similar['Title'][0])
print('title : ',output_fa.text)

title :  Music Copyright for the New Millenium
title :  کپی رایت موسیقی برای هزاره جدید
